# Week 7: Data Aggregation & Pivoting

This week we'll learn how to:

## Part 1: Data Aggregation
- **Basic aggregation** - Counting, summing, averaging data by groups
- **Complex aggregation** - Multiple statistics and advanced grouping
- **Feature engineering** - Creating new variables from aggregated data

## Part 2: Pivoting & Reshaping
- **Pivot tables** - Reshaping data for analysis
- **Melting** - Converting wide data to long format
- **Stacking/Unstacking** - Advanced reshaping techniques

These skills are essential for:
- **Business reporting** - Creating summary statistics and dashboards
- **Data preparation** - Getting data into the right shape for analysis
- **Feature engineering** - Creating predictive variables for machine learning
- **Exploratory analysis** - Understanding patterns in your data

## Your Mission: Analyse LearnTech's Course Performance

You've just been hired as a Data Analyst at **LearnTech**, a growing online education company. Your manager has asked you to analyse course performance and learner behavior to help make important business decisions.

**Your tasks today:**
- **Summarise course performance** - Which courses are most popular? Which have the best completion rates?
- **Understand learner behaviour** - How engaged are our learners? Who are our most successful students?
- **Create business reports** - Present findings in clear, actionable formats
- **Engineer features** - Create learner profiles for personalisation and retention analysis

**What you'll learn:**
- **Data Aggregation** - Turning detailed data into meaningful summaries
- **Pivoting** - Reshaping data for different types of analysis
- **Feature Engineering** - Creating behavioural metrics from transactional data
- **Business Insights** - Translating numbers into actionable recommendations

Don't worry if this seems challenging at first - we'll build up these skills step by step!

## Step 1: Load and Explore Our Data

Let's start by loading our dataset and getting familiar with what we're working with. This dataset contains information about learners and their course enrollments at LearnTech.

In [ ]:
import pandas as pd
import numpy as np

# Load our course data
df = pd.read_csv('online_courses_data.csv')

print(f'Dataset loaded successfully!')
print(f'Dataset size: {df.shape[0]} rows and {df.shape[1]} columns')

df.head()

### Understanding Our Columns

Let's understand what each column represents:
- **learner_id**: Unique identifier for each student
- **course_title**: Name of the course
- **course_subject**: Subject category (Data Science, Web Development, etc.)
- **date_purchased**: When the learner bought the course
- **date_started**: When they began studying
- **date_completed**: When they finished (empty if not completed)
- **percent_complete**: How much of the course they've finished (0-100%)

In [ ]:
# Get basic information about our dataset
print('Column information:')
print(df.info())

print(f'\nQuick overview:')
print(f'Total course enrollments: {len(df):,}')
print(f'Unique learners: {df["learner_id"].nunique():,}')
print(f'Number of courses offered: {df["course_title"].nunique()}')
print(f'Subject areas: {df["course_subject"].nunique()}')

### What Subjects Do We Offer?

In [ ]:
# See what subjects we offer
print('Subjects offered at LearnTech:')
subjects = df['course_subject'].unique()
for subject in subjects:
    print(subject)

## Step 2: Basic Aggregation - Counting Things

Aggregation means **combining multiple rows into summary statistics**. Let's start with the simplest type: counting!

**Business Question:** *Which courses are most popular?*

### Counting Enrollments per Course

Let's count how many students have enrolled in each course. We'll use `.groupby()` to group by course title, then `.size()` to count.

In [ ]:
# Count enrollments per course
enrollments_per_course = df.groupby('course_title').size()

print('Enrollments per course:')
print(enrollments_per_course)

### Making It More Readable

That's a lot of information! Let's sort it to see the most popular courses first.

In [ ]:
# Sort to see most popular courses first
enrollments_sorted = enrollments_per_course.sort_values(ascending=False)

print('Most popular courses:')
print(enrollments_sorted.head(10))

print(f'\nMost popular course: {enrollments_sorted.index[0]}')
print(f'Number of enrollments: {enrollments_sorted.iloc[0]}')

### Understanding .size() vs .count()

You might wonder: why use `.size()` instead of `.count()`? Let's see the difference:
- **`.size()`** counts all rows in each group (including missing values)
- **`.count()`** counts only non-missing values in a specific column

## Step 3: More Complex Aggregation

Now let's answer more complex business questions. We'll build up to using multiple aggregation functions, but first let's try them one at a time to understand what each does.

### Average Progress per Course

**Business Question:** *How much progress do students typically make in each course?*

In [ ]:
# Calculate average progress per course
avg_progress = df.groupby('course_title')['percent_complete'].mean()

print('Average progress per course:')
print(avg_progress.sort_values(ascending=False).head(10).round(1))

### Completion Rates per Course

**Business Question:** *What percentage of students actually finish each course?*

In [ ]:
# First, let's create a completion flag to make this easier
df['is_completed'] = (df['percent_complete'] == 100)

# Now calculate completion rates
completion_rates = df.groupby('course_title')['is_completed'].mean()

print('Completion rates per course:')
print((completion_rates * 100).sort_values(ascending=False).head(10).round(1))

### Total Completions per Course

**Business Question:** *How many students have actually finished each course?*

In [ ]:
# Count total completions per course
total_completions = df.groupby('course_title')['is_completed'].sum()

print('Total completions per course:')
print(total_completions.sort_values(ascending=False).head(10))

### Combining Multiple Aggregations

Now that we understand each aggregation individually, let's combine them using `.agg()`. This is much more efficient than calculating them separately!

In [ ]:
# Combine multiple aggregations using .agg()
course_summary = df.groupby('course_title').agg({
    'learner_id': 'count',           # Total enrollments
    'percent_complete': 'mean',      # Average progress
    'is_completed': ['sum', 'mean']  # Total completions and completion rate
})

print('Combined course statistics:')
course_summary.head()

### Making Column Names Clearer

Those column names are a bit confusing. Let's rename them to be more descriptive.

In [ ]:
# Flatten and rename columns for clarity
course_summary.columns = ['total_enrollments', 'avg_progress', 'total_completions', 'completion_rate']

# Convert completion rate to percentage for readability
course_summary['completion_rate_pct'] = course_summary['completion_rate'] * 100

print('Course summary with clear column names:')
course_summary[['total_enrollments', 'avg_progress', 'completion_rate_pct']].round(1).head()

### Finding the Best and Worst Performers

In [ ]:
# Sort by completion rate to find best and worst performers
best_courses = course_summary.sort_values('completion_rate', ascending=False)

print('Top 5 courses by completion rate:')
best_courses[['total_enrollments', 'completion_rate_pct']].head().round(1)

In [ ]:
# Lets also check the bottom 5 courses by completion rate
print('\nBottom 5 courses by completion rate:')
best_courses[['total_enrollments', 'completion_rate_pct']].tail().round(1)

## Step 4: Feature Engineering - Creating Behavioral Profiles

Now we move from **summarisation** to **feature engineering**. This is a critical skill for data analysts because it transforms transactional data into behavioral insights.

### Why Feature Engineering Matters
When we talk about concepts like "customer behavior" or "employee performance," we're really talking about **patterns across multiple interactions**. A single interactiion may not tell us much, but aggregating many interactions reveals things such as:
- **Frequency patterns** – How often does something occur?
- **Intensity patterns** – How large, strong, or extreme are the typical values?
- **Consistency patterns** – How stable or variable are outcomes over time?
- **Timing patterns** – When do events happen, and are there delays or cycles?
- **Relationship patterns** – How do different variables move together?

This is why aggregation is fundamental to analytics - it creates the features that drive business decisions, personalisation, and predictive models.

### Creating Learner-Level Features

Let's create a comprehensive profile for each learner based on their course-taking behavior. This transforms our course-level data into learner-level insights.

First we need to ensure our date columns are correctly converted to date data types

In [ ]:
# Convert date columns for time-based features
df['date_purchased'] = pd.to_datetime(df['date_purchased'])
df['date_started'] = pd.to_datetime(df['date_started'])
df['date_completed'] = pd.to_datetime(df['date_completed'])

print('Date columns converted to datetime format')

### Understanding the Learner Features

In this step, we’re transforming individual course records into **aggregated learner profiles**.  
Each row in the new dataset represents one learner, summarising their overall engagement and behaviour across all courses they’ve taken.

Here’s what each feature represents and how it’s derived:

| **Feature** | **Description** | **How It’s Calculated** |
|--------------|-----------------|--------------------------|
| **total_courses** | The total number of courses a learner has enrolled in. | Count of all course records per learner (`count` of `course_title`). |
| **subjects_explored** | A measure of how broad the learner’s interests are. | Number of unique subjects taken (`nunique` of `course_subject`). |
| **avg_progress** | The learner’s average completion percentage across courses. | Mean of `percent_complete`. |
| **progress_consistency** | How consistent the learner’s progress is across different courses. A high value means uneven performance, while a low value means similar progress levels. | Standard deviation (`std`) of `percent_complete`. Filled with 0 for learners with only one course. |
| **total_completions** | The total number of courses the learner has finished. | Sum of `is_completed` (where completed courses are marked as 1). |
| **completion_rate** | The proportion of courses completed out of those enrolled. | Mean of `is_completed`. |
| **first_purchase** | The earliest recorded course purchase date. | Minimum (`min`) of `date_purchased`. |
| **last_purchase** | The most recent course purchase date. | Maximum (`max`) of `date_purchased`. |
| **purchase_frequency** | The total number of purchase events. | Count (`count`) of `date_purchased`. |

Together, these features provide a **comprehensive behavioural snapshot** of each learner — capturing how active, consistent, and broad their learning habits are over time.


In [ ]:
# Create comprehensive learner features
learner_features = df.groupby('learner_id').agg({
    'course_title': 'count',                    # Total courses enrolled
    'course_subject': 'nunique',                # Breadth of interests
    'percent_complete': ['mean', 'std'],        # Average progress and consistency
    'is_completed': ['sum', 'mean'],            # Total completions and success rate
    'date_purchased': ['min', 'max', 'count']   # First purchase, last purchase, frequency
})

# Flatten column names
learner_features.columns = ['total_courses', 'subjects_explored', 'avg_progress', 'progress_consistency',
                           'total_completions', 'completion_rate', 'first_purchase', 'last_purchase', 'purchase_frequency']

# Handle edge cases - need to fill in any NaN values in the 'progress_consistency column (std will be NA if only one course)
learner_features['progress_consistency'] = learner_features['progress_consistency'].fillna(0)

print('Learner features created:')
learner_features.head().round(2)

### Engineering Advanced Behavioral Metrics

Now let's create some time based metrics that capture different aspects of learner behaviour. These are the types of features that drive business insights and predictive models.

In [ ]:
# Calculate time-based features

# We will assume the last purchase date in the data is the date of the analysis
analysis_date = df['date_purchased'].max()

# calculate 'days_active' as the number of days between the learners first and last purchase
learner_features['days_active'] = (learner_features['last_purchase'] - learner_features['first_purchase']).dt.days

# calculate 'recency_days' as the number of days since the learner last purchased a course
learner_features['recency_days'] = (analysis_date - learner_features['last_purchase']).dt.days

# check our DataFrame with the added columns
learner_features.head()

### Creating Learner Segments

Using our engineered features, we can now segment learners into meaningful groups. This is how aggregation enables business strategy - turning individual transactions into actionable customer insights.

Here we are simply creating some segments based on some pre-defined rules. In future modules, we will explore how we can exploit machine learning to help with segmentation.

In [ ]:
# Create learner segments based on behavior
def categorise_learner(row):
    if row['completion_rate'] >= 0.8 and row['total_courses'] >= 3:
        return 'High Achiever'
    elif row['completion_rate'] >= 0.5 and row['total_courses'] >= 2:
        return 'Regular Learner'
    elif row['total_courses'] >= 5:
        return 'Course Collector'
    elif row['completion_rate'] < 0.3:
        return 'Struggling Learner'
    else:
        return 'Casual Learner'

learner_features['segment'] = learner_features.apply(categorise_learner, axis=1)

print('Learner segmentation:')
learner_features['segment'].value_counts()

In [ ]:
# Analyse segment characteristics
segment_analysis = learner_features.groupby('segment').agg({
    'total_courses': 'mean',
    'completion_rate': 'mean',
    'avg_progress': 'mean',
    'recency_days': 'mean'
})

print('Segment characteristics:')
segment_analysis.round(2)

## Practice Time: Aggregation Exercises

Now it's your turn! Try to answer these specific business questions using aggregation. Don't worry if you get stuck - we'll work through them together!

### Exercise 1: Most Enrolled Course

**Question:** Which course has the highest number of enrollments?

Hint: Use `.groupby()` and `.size()`. This returns a Pandas series, with column you are grouping by being the index. You can then use `.idxmax()` to return the index of the maximum value and `.max()` to return the maximum value. (Alternatively you could sort the values and just view the first row!)

In [ ]:
# Your code here
# Hint: Use .groupby() and .size(), then find the maximum. 

enrollments = df.groupby('course_title').size()
most_enrolled = enrollments.idxmax()
max_enrollments = enrollments.max()

print(f'Most enrolled course: {most_enrolled}')
print(f'Number of enrollments: {max_enrollments}')

### Exercise 2: Subject with Highest Average Progress

**Question:** Which subject has the highest average progress across all courses?

In [ ]:
# Your code here
# Hint: Group by course_subject and calculate mean of percent_complete

subject_progress = df.groupby('course_subject')['percent_complete'].mean()
best_subject = subject_progress.idxmax()
best_progress = subject_progress.max()

print(f'Subject with highest average progress: {best_subject}')
print(f'Average progress: {best_progress:.1f}%')

### Exercise 3: Course with Shortest Completion Time

**Question:** Among completed courses, which course has the shortest average completion time?

Hint: Filter for completed courses, and then calculate days between start and completion dates. Use `completed = df[df['is_completed']].copy()` to create a copy of the DataFrame which only contains rows where the learner completed the course.

In [ ]:
# Your code here

# Filter for completed courses and calculate completion time
completed = df[df['is_completed']].copy()
completed['days_to_complete'] = (completed['date_completed'] - completed['date_started']).dt.days

# Find course with shortest average completion time
avg_completion_time = completed.groupby('course_title')['days_to_complete'].mean()
fastest_course = avg_completion_time.idxmin()
fastest_time = avg_completion_time.min()

print(f'Fastest course to complete: {fastest_course}')
print(f'Average completion time: {fastest_time:.1f} days')

### Exercise 4: Most Popular Subject

**Question:** Which subject has the most total enrollments?

Hint: Group by course_subject and count enrollments

In [ ]:
# Your code here

subject_enrollments = df.groupby('course_subject').size()
most_popular_subject = subject_enrollments.idxmax()
max_subject_enrollments = subject_enrollments.max()

print(f'Most popular subject: {most_popular_subject}')
print(f'Total enrollments: {max_subject_enrollments}')

### Exercise 5: Subject with Most Unique Learners

**Question:** Which subject attracts the most unique learners? (This might be different from total enrollments!)

Hint: Group by course_subject and count unique learner_ids

In [ ]:
# Your code here

unique_learners_by_subject = df.groupby('course_subject')['learner_id'].nunique()
subject_most_learners = unique_learners_by_subject.idxmax()
max_unique_learners = unique_learners_by_subject.max()

print(f'Subject with most unique learners: {subject_most_learners}')
print(f'Number of unique learners: {max_unique_learners}')

## Step 5: Pivoting - Reshaping Data for Analysis

**Pivoting** is the process of reshaping data between **wide** and **long** formats. This is essential because different types of analysis require different data shapes.

### Understanding Wide vs Long Format

**Long Format (what we usually have):**

| Student | Subject | Grade |
|---------|---------|-------|
| Alice   | Math    | 85    |
| Alice   | Science | 92    |
| Bob     | Math    | 78    |
| Bob     | Science | 88    |


**Wide Format (after pivoting):**

| Student | Math | Science |
|---------|------|--------|
| Alice   | 85   | 92      |
| Bob     | 78   | 88      |


**When to use each format:**
- **Long format**: Good for calculations, filtering, grouping
- **Wide format**: Good for comparisons, correlations, human-readable reports

### Simple Pivot Table

**Business Question:** *How do completion rates vary by subject and year?*

In [ ]:
# First, let's add a year column for easier analysis
df['purchase_year'] = df['date_purchased'].dt.year

print('Years in our dataset:')
print(sorted(df['purchase_year'].unique()))

In [ ]:
# Create a simple pivot table
pivot_completion = df.pivot_table(
    values='is_completed',      # What we want to measure
    index='course_subject',     # Rows
    columns='purchase_year',    # Columns
    aggfunc='mean'             # How to aggregate (average completion rate)
)

print('Completion rates by subject and year:')
(pivot_completion * 100).round(1)  # Convert to percentages

### Understanding Pivot Table Components

Let's break down the components of a pivot table:

- **`values`**: The data you want to summarise (e.g., completion rates, sales amounts)
- **`index`**: What goes in the rows (e.g., products, regions, time periods)
- **`columns`**: What goes in the columns (e.g., years, categories)
- **`aggfunc`**: How to combine multiple values (mean, sum, count, etc.)
- **`fill_value`**: What to show for missing combinations (default is NaN)

This format makes it easy to compare subjects across years at a glance!

### Pivot Table with Counts

Let's create another pivot table showing the number of enrollments by subject and year.

In [ ]:
# Pivot table showing enrollment counts
pivot_enrollments = df.pivot_table(
    values='learner_id',        # What to count
    index='course_subject',     # Rows
    columns='purchase_year',    # Columns
    aggfunc='count',           # Count enrollments
    fill_value=0              # Fill missing values with 0
)

print('Enrollments by subject and year:')
pivot_enrollments

### Adding Totals to Our Pivot Table

In [ ]:
# Add row and column totals
pivot_with_totals = pivot_enrollments.copy()

# Add row totals (total per subject)
pivot_with_totals['Total'] = pivot_with_totals.sum(axis=1)

# Add column totals (total per year)
pivot_with_totals.loc['Total'] = pivot_with_totals.sum(axis=0)

print('Enrollments with totals:')
pivot_with_totals

### Melting - Converting Wide to Long Format

**Melting** is the reverse of pivoting - it converts wide format back to long format. This is useful when:
- You receive data in wide format but need it long for analysis
- You want to create visualisations that require long format
- You need to perform calculations that work better with long data

### Melting Example
Let's say we have this wide data:

| Subject     | 2022 | 2023 |
|-------------|------|------|
| Data Science| 100  | 120  |
| Business    | 80   | 90   |


After melting, it becomes:

| Subject      | Year | Enrollments |
|--------------|------|-------------|
| Data Science | 2022 | 100         |
| Data Science | 2023 | 120         |
| Business     | 2022 | 80          |
| Business     | 2023 | 90          |


In [ ]:
# Convert our pivot table back to long format
melted_data = pivot_enrollments.reset_index().melt(
    id_vars='course_subject',    # Column to keep as identifier
    var_name='year',            # Name for the 'columns' variable
    value_name='enrollments'    # Name for the values
)

print('Melted data (back to long format):')
melted_data.head(10)

### When to Use Melting

Melting is particularly useful for:
1. **Creating visualisations** - Many plotting libraries prefer long format
2. **Statistical analysis** - Most statistical functions expect long format
3. **Database storage** - Normalised databases typically use long format
4. **Further aggregation** - Easier to group and summarize long data

## Practice Time: Pivoting Exercises

Let's practice reshaping data with some specific questions!

### Exercise 6: Monthly Enrollment Trends

**Question:** Create a pivot table showing total enrollments by month (in the rows) and subject (in the columns).

Hint: You'll need to extract month from date_purchased first, you can use: `df['purchase_month'] = df['date_purchased'].dt.month` to do this.

In [ ]:
# Your code here

# Extract month from purchase date
df['purchase_month'] = df['date_purchased'].dt.month

# Create pivot table
monthly_pivot = df.pivot_table(
    values='learner_id',
    index='purchase_month',
    columns='course_subject',
    aggfunc='count',
    fill_value=0
)

print('Monthly enrollments by subject:')
monthly_pivot

### Exercise 7: Average Progress by Subject and Year

**Question:** Create a pivot table showing average progress by subject and year.

Hint: Use `percent_complete` as values and `'mean'` as `aggfunc`

In [ ]:
# Your code here

progress_pivot = df.pivot_table(
    values='percent_complete',
    index='course_subject',
    columns='purchase_year',
    aggfunc='mean'
)

print('Average progress by subject and year:')
progress_pivot.round(1)

### Exercise 8: Subject Performance Data

**Question:** Create a DataFrame which summarises the number of courses each learner has purchased within each subject (the subjects should form the columns). Add a totals column.

In [ ]:
# Your code here
# Hint: This is similar to the first pivot table we created

subject_performance_data = df.pivot_table(
    values='course_title',
    index='learner_id',
    columns='course_subject',
    aggfunc='count',
    fill_value=0
)

subject_performance_data['total'] = subject_performance_data.sum(axis=1)

subject_performance_data.sort_values(by='total',ascending=False).head()

## Congratulations! You've Mastered Aggregation & Pivoting

Great work! You've successfully completed your first major data analysis project at LearnTech. Let's recap what you've accomplished:

### Aggregation Skills You've Learned:
- **Basic counting** with `.groupby()` and `.size()`
- **Statistical summaries** using `.mean()`, `.sum()`, etc.
- **Multiple aggregations** with `.agg()` for comprehensive analysis
- **Feature engineering** - Creating behavioral profiles from transactional data
- **Business segmentation** - Turning features into actionable customer insights

### Pivoting Skills You've Learned:
- **Creating pivot tables** with `.pivot_table()`
- **Understanding wide vs long format** and when to use each
- **Cross-tabulation** for comparing categories
- **Melting data** to convert between formats
- **Data reshaping** for different types of analysis

### Key Concepts for Data Analysts:
- **Aggregation drives insights** - Individual transactions become behavioural patterns
- **Feature engineering is critical** - Raw data rarely contains the metrics you need
- **Data shape matters** - Different analyses require different formats
- **Business context is key** - Technical skills must serve business objectives

### Business Value You've Created:
- Identified most and least popular courses
- Found subjects with highest completion rates
- Created learner segments for personalisation
- Discovered trends over time
- Built actionable insights for course improvement

### What's Next:
These skills form the foundation for more advanced analytics:
- **Customer lifetime value** calculations
- **Churn prediction** models
- **Recommendation systems**
- **A/B testing** analysis
- **Time series forecasting**
